# Building Effective Agents (with Pydantic AI)

Examples for the agentic workflows discussed in
[Building Effective Agents](https://www.anthropic.com/research/building-effective-agents)
by [Erik Schluntz](https://github.com/eschluntz) and [Barry Zhang](https://github.com/ItsBarryZ)
of Anthropic, inspired, ported and adapted from the
[code samples](https://github.com/anthropics/anthropic-cookbook/tree/main/patterns/agents)
by the authors using [Pydantic AI](https://ai.pydantic.dev/).

## Basic Workflows
Examples copied from [Intellectronica - Building Effective Agents with Pydantic AI](https://github.com/intellectronica/building-effective-agents-with-pydantic-ai)

In [4]:
%pip install -r requirements.txt
from IPython.display import clear_output ; clear_output()

In [5]:
# Import and initialize the AI model
from util import initialize, show
AI_MODEL = initialize()

import asyncio
from typing import List, Dict

from pydantic import BaseModel, Field
from pydantic_ai import Agent

print(f"\n✅ Successfully initialized with model type: {type(AI_MODEL)}")
if hasattr(AI_MODEL, 'model_name'):
    print(f"🤖 Using Ollama model via OpenAI-compatible interface")
else:
    print(f"🤖 Using standard model: {AI_MODEL}")

Attempting to instrument while already instrumented


Available AI models:
['azure:gpt-4o', 'azure:gpt-4o-mini']

Using AI model: azure:gpt-4o
Configuring Azure AI Foundry model: gpt-4o at https://agent-workshop-yrkd.cognitiveservices.azure.com/

✅ Successfully initialized with model type: <class 'pydantic_ai.models.openai.OpenAIModel'>
🤖 Using Ollama model via OpenAI-compatible interface


### Workflow: Prompt Chaining

The first and simplest workflow, chaining, we feed the output of one LLM
call to the next one, and complete our task step by step.

> <img src="https://ai.pydantic.dev/img/pydantic-ai-dark.svg" style="height: 1em;" />
> All LLM calls are handled by the `Agent` class. One feature we take advantage of
> here is a consistent interface for different LLM providers. The model name is
> provided to the constructor and from there, the calls are the same regardless of
> which provider and model we are using.

### 🚀 Ollama Integration

This notebook now supports **Ollama** models alongside OpenAI, Anthropic, and Google models! 

#### How it works:
- **Automatic Detection**: The `util.py` automatically detects available Ollama models by running `ollama list`
- **OpenAI-Compatible Interface**: Ollama models are accessed through Pydantic AI's OpenAI-compatible interface
- **Seamless Integration**: Use `Agent(AI_MODEL)` exactly the same way regardless of the underlying model

#### Configuration:
- **Automatic**: Just have Ollama running with models installed
- **Manual**: Set `AI_MODEL=ollama:model-name` in your `.env` file
- **Custom URL**: Set `OLLAMA_BASE_URL` if using a remote Ollama server

#### Available Models:
The system automatically detects all your locally installed Ollama models. Popular choices include:
- `llama3.2` - Meta's latest Llama model
- `mistral` - Mistral AI's efficient model  
- `deepseek-coder` - Specialized coding model
- `phi4-mini` - Microsoft's compact model

> 💡 **Tip**: You can switch between any supported model by setting the `AI_MODEL` environment variable!

### 🌐 Azure AI Foundry Integration

This notebook now supports **Azure AI Foundry** models alongside OpenAI, Anthropic, Google, and Ollama models! 

#### How it works:
- **Seamless Integration**: Azure models are accessed through Pydantic AI's Azure provider
- **Automatic Configuration**: The `util.py` automatically configures Azure models when `azure:` prefix is used
- **Enterprise Ready**: Perfect for enterprise deployments with Azure compliance and security

#### Configuration:
1. **Create Azure AI Foundry Project**: Visit [Azure AI Foundry](https://ai.azure.com/) and create a project
2. **Deploy Models**: Deploy models like GPT-4o or GPT-4o-mini in your project
3. **Get Credentials**: Find your API key and endpoint in project settings
4. **Set Environment Variables**:
   ```bash
   AZURE_API_KEY=your-azure-api-key
   AZURE_ENDPOINT=https://your-resource.openai.azure.com/
   AZURE_API_VERSION=2024-08-01-preview
   AI_MODEL=azure:gpt-4o
   ```

#### Available Models:
Azure AI Foundry supports various OpenAI models through deployment:
- `gpt-4o` - Latest GPT-4 Optimized model
- `gpt-4o-mini` - Faster, cost-effective GPT-4 variant
- `gpt-4-turbo` - High-performance GPT-4 model
- `gpt-3.5-turbo` - Efficient GPT-3.5 model

> 💡 **Enterprise Tip**: Azure AI Foundry provides enterprise-grade security, compliance, and data residency controls perfect for production deployments!

In [6]:
# Example of Azure AI Foundry configuration (for reference only)
# This shows how Azure models are configured internally by util.py

"""
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.azure import AzureProvider

# Azure AI Foundry model configuration
azure_model = OpenAIModel(
    model_name='gpt-4o',  # Your deployed model name
    provider=AzureProvider(
        azure_endpoint='https://your-resource.openai.azure.com/',
        api_version='2024-08-01-preview',
        api_key='your-azure-api-key'
    )
)

# Use with Agent
from pydantic_ai import Agent
agent = Agent(azure_model)
"""

print("💡 Azure AI Foundry models are automatically configured when using 'azure:' prefix in AI_MODEL")
print("🔧 The util.py handles all the Azure provider setup for you!")

if hasattr(AI_MODEL, 'provider') and 'Azure' in str(type(AI_MODEL.provider)):
    print(f"✅ Currently using Azure AI Foundry model: {AI_MODEL.model_name}")
    print(f"🌐 Provider: {type(AI_MODEL.provider).__name__}")
else:
    print(f"ℹ️  Current model: {AI_MODEL} (not Azure)")
    print("   To use Azure, set AI_MODEL=azure:gpt-4o in your .env file")

💡 Azure AI Foundry models are automatically configured when using 'azure:' prefix in AI_MODEL
🔧 The util.py handles all the Azure provider setup for you!
ℹ️  Current model: OpenAIModel() (not Azure)
   To use Azure, set AI_MODEL=azure:gpt-4o in your .env file


In [7]:
async def chain(input: str, prompts: List[str]) -> str:
    """Chain multiple LLM calls sequentially, passing results between steps."""
    agent = Agent(AI_MODEL)
    result = input
    for i, prompt in enumerate(prompts, 1):
        agent = Agent(AI_MODEL, system_prompt=prompt)
        response = await agent.run(f"nInput:\n{result}")
        result = response.output
        show(result, title=f"Step {i}")
    return result

In [8]:
data_processing_steps = [
    """Extract only the numerical values and their associated minion metrics from the text.
    Format each as 'value: minion metric' on a new line.
    Example format:
    92: banana consumption rate
    45%: "BELLO!" frequency increase""",
    
    """Convert all numerical values to percentages where possible for minion analytics.
    If not a percentage or points, convert to decimal (e.g., 92 bananas -> 92%).
    Keep one number per line.
    Example format:
    92%: banana consumption rate
    45%: "BELLO!" frequency increase""",
    
    """Sort all lines in descending order by numerical value for minion performance ranking.
    Keep the format 'value: minion metric' on each line.
    Example:
    92%: banana consumption rate
    87%: yellow outfit compliance""",
    
    """Format the sorted minion data as a markdown table with columns:
    | Minion Metric | Value |
    |:--|--:|
    | Banana Consumption Rate | 92% |"""
]

report = """
Q3 Minion Performance Summary:
Our minion happiness score rose to 92 points this quarter after the banana supply increase.
"BELLO!" frequency grew by 45% compared to last year's metrics.
Yellow outfit compliance is now at 23% improvement in our primary lab.
Minion distraction incidents decreased to 5% from 8% after removing shiny objects.
New evil scheme participation cost is $43 per minion recruit.
Gadget adoption rate increased to 78% among tech-savvy minions.
Gru satisfaction with minions is at 87 points.
Banana storage efficiency improved to 34% reduction in spoilage.
""".strip()

show(report, title="Input text")
formatted_result = await chain(report, data_processing_steps)
show(formatted_result, title="Result")


Input text
----------

Q3 Minion Performance Summary:
Our minion happiness score rose to 92 points this quarter after the banana supply increase.
"BELLO!" frequency grew by 45% compared to last year's metrics.
Yellow outfit compliance is now at 23% improvement in our primary lab.
Minion distraction incidents decreased to 5% from 8% after removing shiny objects.
New evil scheme participation cost is $43 per minion recruit.
Gadget adoption rate increased to 78% among tech-savvy minions.
Gru satisfaction with minions is at 87 points.
Banana storage efficiency improved to 34% reduction in spoilage.


Step 1
------

92: minion happiness score  
45%: "BELLO!" frequency increase  
23%: yellow outfit compliance improvement  
5%: minion distraction incidents  
$43: evil scheme participation cost  
78%: gadget adoption rate  
87: Gru satisfaction with minions  
34%: banana storage efficiency improvement


Step 2
------

92%: minion happiness score  
45%: "BELLO!" frequency increase  
23%: yello

### Workflow: Routing

In the routing workflow, we rely on an LLM call to decide which
of several options to take. An AI agent is now handling part of
the logic of the app.

> <img src="https://ai.pydantic.dev/img/pydantic-ai-dark.svg" style="height: 1em;" />
> One of the most powerful features of Pydantic AI is ... Pydantic!
> Returing results using structured outputs and validating them against
> the schema specified is built-in. We can specify a `result_type`
> for every agent and receive the parsed and validated Pydantic model
> instance from the LLM call. This is easily one of the best ways of
> interfacing between code and LLM calls, and we'll be using it for
> most workflows.

In [9]:
class RouteSelection(BaseModel):
    reasoning: str = Field(..., description=(
        'Brief explanation of why this ticket should be routed '
        'to a specific team. Consider key terms, user intent, '
        'and urgency level.'
    ))
    selection: str = Field(..., description='The chosen team name')


async def route(input: str, routes: Dict[str, str]) -> str:
    """Route input to specialized prompt using content classification."""

    # First, determine appropriate route using LLM with chain-of-thought
    show(f"{list(routes.keys())}", title="Available Routes")
    
    routing_agent = Agent(
        AI_MODEL,
        system_prompt=(
            'Analyze the input and select the most appropriate support team '
            f'from these options: {list(routes.keys())}'
        ),
        output_type=RouteSelection,
    )
    route_response = await routing_agent.run(input)
    reasoning = route_response.output.reasoning
    route_key = route_response.output.selection.strip().lower()
    
    show(reasoning, title="Routing Analysis")
    show(f"{route_key}", title="Selected Route")
    
    # Process input with selected specialized prompt
    worker_agent = Agent(AI_MODEL, system_prompt=routes[route_key])
    return (await worker_agent.run(input)).output

In [10]:
support_routes = {
    "banana": """You are a banana supply specialist at Gru's lab. Follow these guidelines:
    1. Always start with "Banana Supply Response:"
    2. First acknowledge the specific banana-related issue
    3. Explain banana inventory, quality, or delivery problems clearly
    4. List concrete next steps with timeline for banana resolution
    5. End with alternative banana options if relevant
    
    Keep responses enthusiastic but professional about banana matters.""",
    
    "gadget": """You are a minion gadget technical support engineer. Follow these guidelines:
    1. Always start with "Gadget Support Response:"
    2. List exact steps to resolve the gadget malfunction
    3. Include safety requirements for minion operation
    4. Provide workarounds for common gadget problems
    5. End with escalation path to Dr. Nefario if needed
    
    Use clear, numbered steps and mention "BELLO!" for encouragement.""",
    
    "security": """You are a minion security specialist for Gru's operations. Follow these guidelines:
    1. Always start with "Security Support Response:"
    2. Prioritize lab security and minion identification verification
    3. Provide clear steps for access recovery/changes to restricted areas
    4. Include security tips and warnings about minion imposters
    5. Set clear expectations for resolution time
    
    Maintain a serious, security-focused tone while being minion-friendly.""",
    
    "training": """You are a minion training specialist. Follow these guidelines:
    1. Always start with "Training Support Response:"
    2. Focus on minion skill development and best practices
    3. Include specific examples of proper minion behavior
    4. Link to relevant training manual sections
    5. Suggest related skills that might help with evil schemes
    
    Be educational and encouraging, use minion language occasionally."""
}

# Test with different minion support tickets
tickets = [
    """Subject: Can't access the secret lab
    Message: BELLO! I've been trying to get into the secret lab for the past hour but the scanner 
    keeps saying 'unrecognized minion' error. I'm sure I'm the right minion! Can you help me regain 
    access? This is urgent as I need to prepare the freeze ray by end of day.
    - Minion Kevin""",
    
    """Subject: Banana shortage in cafeteria
    Message: Hello, I just noticed we're completely out of bananas in the minion cafeteria, but I 
    thought we had a fresh shipment scheduled for today. The other minions are getting restless 
    without their banana breaks. Can you explain this shortage and fix it?
    Thanks,
    Minion Stuart""",
    
    """Subject: How to operate the new shrink ray?
    Message: I need to learn how to operate the new shrink ray gadget for tomorrow's mission. I've 
    looked through the manual but can't figure out the safety protocols. Is there a training 
    session available? Could you walk me through the steps?
    Best regards,
    Minion Bob"""
]

print("Processing minion support tickets...\n")
for i, ticket in enumerate(tickets, 1):
    show(ticket, title=f"Ticket {i}")
    response = await route(ticket, support_routes)
    show(response, title=f"Response {i}")

Processing minion support tickets...


Ticket 1
--------

Subject: Can't access the secret lab
    Message: BELLO! I've been trying to get into the secret lab for the past hour but the scanner 
    keeps saying 'unrecognized minion' error. I'm sure I'm the right minion! Can you help me regain 
    access? This is urgent as I need to prepare the freeze ray by end of day.
    - Minion Kevin


Available Routes
----------------

['banana', 'gadget', 'security', 'training']


Routing Analysis
----------------

The issue described involves access problems which are characterized by the 'unrecognized minion' error. This suggests an authentication or identification issue, which falls under security concerns. The urgency of needing access to prepare the freeze ray further emphasizes the need for prompt security intervention.


Selected Route
--------------

security


Response 1
----------

Security Support Response:

Attention Minion Kevin,

Firstly, your efforts to ensure timely project compl

### Workflow: Parallelization

LLM calls are typically long-running, I/O-bound operations. They are also stateless,
so in cases where one call doesn't depend on the other, it makes a lot of sense
to parallelize the calls and continue processing once we receive the responses
from all of them.

> <img src="https://ai.pydantic.dev/img/pydantic-ai-dark.svg" style="height: 1em;" />
> `Agent.run()` is an asynchronous call, and in the code here is asnchronous by default.
> Pydantic AI also provides an `Agent.run_sync()` method for cases where you're writing
> synchronous code.


In [11]:
async def parallel(prompt: str, inputs: List[str]) -> List[str]:
    """Process multiple inputs concurrently with the same prompt."""
    agent = Agent(AI_MODEL, system_prompt=prompt)
    results = await asyncio.gather(*[
        agent.run(input)
        for input in inputs
    ])
    return [result.output for result in results]

In [12]:
stakeholders = [
    """Minions:
    - Banana supply dependent
    - Want latest gadgets
    - Safety concerns with explosions""",
    
    """Lab Scientists (Dr. Nefario):
    - Equipment reliability worries
    - Need advanced technology
    - Want clear evil scheme direction""",
    
    """Gru (Boss):
    - Expects successful missions
    - Want cost-effective operations
    - Reputation risks with failed schemes""",
    
    """Gadget Suppliers:
    - Production capacity constraints
    - Price pressures from competition
    - Technology upgrade transitions"""
]

impact_results = await parallel(
    """Analyze how changes in the evil villain industry will impact this stakeholder group.
    Provide specific impacts and recommended actions for Gru's operations.
    Format with clear sections and priorities.""",
    stakeholders
)

for stakeholder, result in zip(stakeholders, impact_results):
    show(result, stakeholder.split(':')[0])


Minions
-------

**Analysis of Changes in the Evil Villain Industry and Their Impact on Minions**

In recent years, the evil villain industry has witnessed significant changes, including technological advancements, heightened security regulations, and more emphasis on ethical operations. These transformations will impact the minions, a crucial stakeholder group in Gru's operations, in various ways. Below is a detailed analysis of these impacts, along with recommended actions prioritized to ensure a smooth transition and maintain minion satisfaction and efficiency.

### Impact on Minions

1. **Banana Supply Chain Disruptions**
   - **Impact:** The heightened focus on sustainability and ethical sourcing in global markets may lead to disruptions in the banana supply chain. Minions, being highly dependent on bananas for energy and motivation, could face shortages, thereby affecting their productivity and morale.
   - **Recommended Action:** Gru should consider securing alliances with sust